# VacationPy
----

#### Final Report

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os
from pprint import pprint
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv('../output_data/weather_data.csv')
weather_df.head(1)

,City,Latitude,Longitude,MaxTemp (°C),Humidity (%),Cloudiness (%),WindSpeed (meter/sec),Country,Date
0,Laas,46.62,10.7,8.0,87,40,3.6,IT,06/18/20


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with coordinates and humidity as weight
gmaps.configure(g_key)
fig = gmaps.figure(zoom_level=2, center=(30,70))
heatmap_layer = gmaps.heatmap_layer(
   weather_df[['Latitude', 'Longitude']], weights=weather_df['Humidity (%)'],
    max_intensity=100, dissipating=False, point_radius=3.0)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
gmaps.configure(g_key)
fig = gmaps.figure(map_type='HYBRID', zoom_level=4, center=(10,100))
heatmap_layer = gmaps.heatmap_layer(
   weather_df[['Latitude', 'Longitude']], weights=weather_df['Humidity (%)'],
    max_intensity=100, dissipating=False, point_radius=3.0)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame with cities I plan to visit and weather criteria

In [5]:
from api_keys import weather_api_key
# Buil query url
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "metric"
query_url = f"{url}appid={weather_api_key}&units={units}&q="

In [6]:
# Thinking about going on vacation to these cities
cities = ["El Nido", "Coron", "Phuket", "Seoul", "Tokyo", "Maldives", "Krabi", "Singapore", "Kuala Lumpur", "Taipei", "Hanoi"]
# Set up empty lists to hold weather data  
destination = []
latitude = []
longitude = []
temp = []
windspeed = []
cloudiness = []
country =[]
# Loop through cities and perform a request for json data
for city in cities:
    response = requests.get(query_url + city).json()
    destination.append(response['name'])
    latitude.append(response['coord']['lat'])
    longitude.append(response['coord']['lon'])
    temp.append(response['main']['temp_max'])
    windspeed.append(response['wind']['speed'])
    cloudiness.append(response['clouds']['all'])
    country.append(response['sys']['country'])

In [7]:
# Create dataframe of planned tourist destinations
destination_dict = {"City": destination, "Lat": latitude, "Lng": longitude, "Temp (°C)": temp, "Windspeed (mps)": windspeed, "Cloudiness": cloudiness, "Country": country}
planned_cities = pd.DataFrame(destination_dict)
planned_cities

,City,Lat,Lng,Temp (°C),Windspeed (mps),Cloudiness,Country
0,El Nido,11.19,119.40,28.51,1.88,77,PH
1,Coron,12.00,120.20,26.76,1.06,96,PH
2,Phuket,8.00,98.25,26.00,0.50,20,TH
3,Seoul,37.57,126.98,23.00,1.50,0,KR
4,Tokyo,35.69,139.69,21.11,5.10,75,JP
5,Maldives,3.20,73.00,28.78,4.97,30,MV
6,Krabi,8.07,98.91,27.22,1.50,40,TH
7,Singapore,1.29,103.85,27.00,4.02,75,SG
8,Kuala Lumpur,3.14,101.69,27.00,4.60,40,MY
9,Taipei,25.05,121.53,30.00,2.10,90,TW


In [8]:
# Filter dataframe that fits to my personal weather criteria (metric units)
best_vacations = planned_cities.loc[(planned_cities['Temp (°C)'] <29) & (planned_cities['Temp (°C)'] >=26)
                            & (planned_cities['Windspeed (mps)'] < 10) & (planned_cities['Cloudiness'] < 90)]
# Sort results by ascending temp
#best_vacations.sort_values('Temp (°C)', ascending=True, inplace=True)
best_vacations

,City,Lat,Lng,Temp (°C),Windspeed (mps),Cloudiness,Country
0,El Nido,11.19,119.40,28.51,1.88,77,PH
2,Phuket,8.00,98.25,26.00,0.50,20,TH
5,Maldives,3.20,73.00,28.78,4.97,30,MV
6,Krabi,8.07,98.91,27.22,1.50,40,TH
7,Singapore,1.29,103.85,27.00,4.02,75,SG
8,Kuala Lumpur,3.14,101.69,27.00,4.60,40,MY


### How about we look for more vacation spots based from the random city data of my WeatherPy CSV file, and drop cities that don't fit my criteria? 

In [9]:
# Using prior activity's csv output
other_options = weather_df.loc[(weather_df['MaxTemp (°C)'] <29) & (weather_df['MaxTemp (°C)'] >=26)
                            & (weather_df['WindSpeed (meter/sec)'] < 10) & (weather_df['WindSpeed (meter/sec)'] < 90)]
other_options.head()

,City,Latitude,Longitude,MaxTemp (°C),Humidity (%),Cloudiness (%),WindSpeed (meter/sec),Country,Date
4,San Patricio,19.22,-104.70,26.49,64,17,1.31,MX,06/18/20
21,Yangi Marg`ilon,40.43,71.72,26.00,22,88,3.10,UZ,06/18/20
22,Lahaina,20.88,-156.68,26.00,65,20,7.20,US,06/18/20
38,Ambilobe,-13.20,49.05,26.81,40,0,1.09,MG,06/18/20
49,Kapaa,22.08,-159.32,26.00,78,90,8.20,US,06/18/20


In [10]:
# Drop any NaN values then check 
other_options = other_options.dropna()
other_options.isna().sum()

City                     0
Latitude                 0
Longitude                0
MaxTemp (°C)             0
Humidity (%)             0
Cloudiness (%)           0
WindSpeed (meter/sec)    0
Country                  0
Date                     0
dtype: int64

### Hotel Map
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
# Adding a column to my planned destinations dataframe for hotel names 
with_hotel = planned_cities
with_hotel['Hotel Name'] = " "
with_hotel.reset_index(drop=True)
with_hotel.head(1)

,City,Lat,Lng,Temp (°C),Windspeed (mps),Cloudiness,Country,Hotel Name
0,El Nido,11.19,119.4,28.51,1.88,77,PH,


In [15]:
# Set parameters for Google Places API 
params = {'key': g_key,
          'keyword': 'hotel',
         'radius': 5000,
          'type': 'lodging'}

# Loop using the city's coordinates
for index, row in with_hotel.iterrows():
    search_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?key" + g_key
    Latitude = row['Lat']
    Longitude = row['Lng']
#     Update location each iteration
    params['location'] = f"{Latitude},{Longitude}"
#     Make request with parameters
    hotel_name = requests.get(search_url, params=params)
#     Convert to json
    hotel_name = hotel_name.json()
    #check results
    #pprint(json.dumps(hotel_name))
    
#     add the first item on the list of search results
    with_hotel.loc[index, "Hotel Name"] = hotel_name['results'][0]['name']    

In [16]:
with_hotel.head(5)

,City,Lat,Lng,Temp (°C),Windspeed (mps),Cloudiness,Country,Hotel Name
0,El Nido,11.19,119.40,28.51,1.88,77,PH,El Nido Mahogany Beach Resort
1,Coron,12.00,120.20,26.76,1.06,96,PH,Sunz En Coron Resort
2,Phuket,8.00,98.25,26.00,0.50,20,TH,Twinpalms Phuket Resort
3,Seoul,37.57,126.98,23.00,1.50,0,KR,LOTTE HOTEL SEOUL
4,Tokyo,35.69,139.69,21.11,5.10,75,JP,The Okura Tokyo


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [51]:
# Create dict using hotel name and city coordinates from the previous dataframe
hotels = [
    {'name': 'El Nido Mahogany Beach Resort', 'city': 'El Nido', 'country': 'PH', 'location': (11.19, 119.40)},
    {'name': 'Sunz En Coron Resort', 'city': 'Coron', 'country': 'PH', 'location': (12.00, 120.20)},
    {'name': 'Twinpalms Phuket Resort', 'city': 'Phuket', 'country': 'PH', 'location': (8.00, 98.25)},
    {'name': 'LOTTE HOTEL SEOUL', 'city': 'Seol', 'country': 'KR', 'location': (37.57, 126.98)},
    {'name': 'The Okura Tokyo', 'city': 'Tokyo', 'country': 'JP', 'location': (35.69, 139.69)}
]

hotel_location = [hotel['location'] for hotel in hotels]

info_box_template = """
<dl>
<dt>Name</dt><dd>{name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""

hotel_info = [info_box_template.format(**hotel) for hotel in hotels]

marker_layer = gmaps.marker_layer(hotel_location, info_box_content=hotel_info)

# Customize the size of figure
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, map_type='HYBRID', zoom_level=4, center=(25,115))

fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…